# Loading libs

In [ ]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from sklearn.metrics import classification_report
import tensorflow as tf
import cv2
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import zipfile

<a id = "1"></a><br>
# Load Dataset

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
#downloading dataset
!kaggle datasets download -d arunrk7/surface-crack-detection

 98% 229M/233M [00:01<00:00, 174MB/s]
100% 233M/233M [00:01<00:00, 149MB/s]


In [ ]:
#unzipping the dataset and deleting the zipped file
zip_path = '/content/surface-crack-detection.zip'
data_dir = '/content/uncompressed'

os.makedirs(data_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)
#RAM reasons
os.remove(zip_path)


# deleting half the database because the RAM couldn't take it

In [ ]:
import glob
#definig the function
def delete_half_files(path):
    files = glob.glob(path + '/*')

    half = len(files) // 2

    for i in range(half):
        os.remove(files[i])

In [ ]:
delete_half_files('/content/uncompressed/Negative')
delete_half_files('/content/uncompressed/Positive')

# Transforming the images into a dataset

In [ ]:
img_size = 120
def read_images(data_dir, labels=['Negative', 'Positive'], img_size=120):
    data = []
    for label in labels:
        for img in os.listdir(os.path.join(data_dir, label)):
            try:
                img_arr = cv2.imread(os.path.join(data_dir, label, img), cv2.IMREAD_GRAYSCALE)
                data.append([cv2.resize(img_arr, (img_size, img_size)), labels.index(label)])
            except Exception as e:
                print(e)
    return np.array(data)

Dataset = read_images('/content/uncompressed')

deleting the directory because of the RAM..again.

In [ ]:
import shutil
shutil.rmtree(data_dir)

<a id = "3"></a><br>
# Data preprocessing

In [ ]:
#transform into an array
x = []
y = []

for feature, label in Dataset:
    x.append(feature)
    y.append(label)

x = np.array(x).reshape(-1, img_size, img_size, 1)
x = x / 255
y = np.array(y)

In [ ]:
#RAM strikes again
del Dataset

In [ ]:
#splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size = 0.25 ,random_state=0)#default splitting 0.25

In [ ]:
#yarab
del x
del y

## Model training

In [ ]:
#writing the model
num_of_classes = 2
model = Sequential(
    [
Conv2D(64,3,padding="same", activation="relu", input_shape = X_train.shape[1:] ),
MaxPool2D( ),

Conv2D(64, 3, padding="same", activation="relu" ),
MaxPool2D( ),

Conv2D(128, 3, padding="same", activation="relu" ),
MaxPool2D(),

Flatten(),
Dense(256,activation="relu"),
Dropout(0.5),
BatchNormalization(),
Dense(num_of_classes, activation="softmax"),
    ]

                  )



In [ ]:

#the training
opt = Adam(learning_rate=1e-5)

model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

model.fit(X_train, y_train, epochs = 15, batch_size = 128, validation_split = 0.25)

Epoch 1/15
88/88 [==============================] - 339s 4s/step - loss: 0.2120 - accuracy: 0.8999 - val_loss: 0.1074 - val_accuracy: 0.9872
Epoch 2/15
88/88 [==============================] - 338s 4s/step - loss: 0.0451 - accuracy: 0.9849 - val_loss: 0.1084 - val_accuracy: 0.9757
Epoch 3/15
88/88 [==============================] - 324s 4s/step - loss: 0.0321 - accuracy: 0.9908 - val_loss: 0.0549 - val_accuracy: 0.9848
Epoch 4/15
88/88 [==============================] - 325s 4s/step - loss: 0.0261 - accuracy: 0.9921 - val_loss: 0.0613 - val_accuracy: 0.9768
Epoch 5/15
88/88 [==============================] - 328s 4s/step - loss: 0.0205 - accuracy: 0.9938 - val_loss: 0.0270 - val_accuracy: 0.9939
Epoch 6/15
88/88 [==============================] - 334s 4s/step - loss: 0.0193 - accuracy: 0.9941 - val_loss: 0.0360 - val_accuracy: 0.9909
Epoch 7/15
88/88 [==============================] - 327s 4s/step - loss: 0.0158 - accuracy: 0.9953 - val_loss: 0.0209 - val_accuracy: 0.9939
Epoch 8/15
88

# Evaluation

In [ ]:
model.evaluate(X_test,y_test)

157/157 [==============================] - 37s 234ms/step - loss: 0.0207 - accuracy: 0.9954


[0.020732024684548378, 0.9954000115394592]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 60, 60, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 128)       0